Importing the Dependencies

In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score


Data Collection and Analysis

PIMA Diabetes Dataset

In [58]:
# loading the diabetes dataset to a pandas DataFrame
diabetes_dataset = pd.read_csv('./Datasets/diabetes.csv')

In [59]:
pd.read_csv?

In [60]:
# printing the first 5 rows of the dataset
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [61]:
# number of rows and Columns in this dataset
diabetes_dataset.shape

(768, 9)

In [62]:
# getting the statistical measures of the data
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [63]:
diabetes_dataset['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

0 --> Non-Diabetic

1 --> Diabetic

In [64]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


Check Normalization and Standarization


In [65]:

def check_data_distribution(df: pd.DataFrame, alpha=0.05):
    """
                *checking data is Std or Normalization*

                *Parameters*
                df = input datasets
                alpha = state shapiro exam parameter

                *Output*
                discription any colum parameter dataset

    """

    report = []
    for col in df.columns:
        data = df[col].dropna()
        mean = data.mean()
        std = data.std()
        min_ = data.min()
        max_ = data.max()

        # استاندارد بودن (میانگین نزدیک صفر و std نزدیک 1)
        is_standardized = (abs(mean) < 0.1) and (abs(std - 1) < 0.1)

        # نرمال‌سازی شده (مقادیر بین 0 و 1)
        is_normalized = (min_ >= 0) and (max_ <= 1)

        # آزمون Shapiro-Wilk
        if len(data) >= 3 and len(data) <= 5000:  # محدودیت shapiro
            stat, p = shapiro(data)
            is_normal_dist = (p > alpha)
        else:
            is_normal_dist = None  # داده خیلی زیاد یا کم است

        report.append({
            'column': col,
            'mean': mean,
            'std': std,
            'min': min_,
            'max': max_,
            'is_standardized': is_standardized,
            'is_normalized': is_normalized,
            'shapiro_normal_dist': is_normal_dist
        })

    return pd.DataFrame(report)


In [66]:

report = check_data_distribution(diabetes_dataset)
print(report)

                     column        mean         std     min     max  \
0               Pregnancies    3.845052    3.369578   0.000   17.00   
1                   Glucose  120.894531   31.972618   0.000  199.00   
2             BloodPressure   69.105469   19.355807   0.000  122.00   
3             SkinThickness   20.536458   15.952218   0.000   99.00   
4                   Insulin   79.799479  115.244002   0.000  846.00   
5                       BMI   31.992578    7.884160   0.000   67.10   
6  DiabetesPedigreeFunction    0.471876    0.331329   0.078    2.42   
7                       Age   33.240885   11.760232  21.000   81.00   
8                   Outcome    0.348958    0.476951   0.000    1.00   

   is_standardized  is_normalized  shapiro_normal_dist  
0            False          False                False  
1            False          False                False  
2            False          False                False  
3            False          False                False  
4   

In [67]:
# separating the data and labels
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
Y = diabetes_dataset['Outcome']

In [68]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [69]:
print(Y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


Data Standardization

In [70]:
scaler = StandardScaler()

In [71]:
scaler.fit(X)

,copy,True
,with_mean,True
,with_std,True


In [72]:
standardized_data = scaler.transform(X)

In [73]:
print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [74]:
# X = standardized_data
Y = diabetes_dataset['Outcome']

In [75]:
print(X)
print(Y)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


check normalization after a fit transform for X data



In [76]:
X = pd.DataFrame(standardized_data, columns=X.columns)
report = check_data_distribution(X)
print(report)

                     column          mean       std       min       max  \
0               Pregnancies -6.476301e-17  1.000652 -1.141852  3.906578   
1                   Glucose -9.251859e-18  1.000652 -3.783654  2.444478   
2             BloodPressure  1.503427e-17  1.000652 -3.572597  2.734528   
3             SkinThickness  1.006140e-16  1.000652 -1.288212  4.921866   
4                   Insulin -3.006854e-17  1.000652 -0.692891  6.652839   
5                       BMI  2.590520e-16  1.000652 -4.060474  4.455807   
6  DiabetesPedigreeFunction  2.451743e-16  1.000652 -1.189553  5.883565   
7                       Age  1.931325e-16  1.000652 -1.041549  4.063716   

   is_standardized  is_normalized  shapiro_normal_dist  
0             True          False                False  
1             True          False                False  
2             True          False                False  
3             True          False                False  
4             True          False      

Train Test Split

In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [78]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


testing acc by Model

In [79]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
models = {
    'Linear SVM': svm.SVC(kernel='linear'),
    'RBF SVM': svm.SVC(kernel='rbf'),
    'KNN': KNeighborsClassifier(),
    'Neural Net': MLPClassifier(max_iter=1000)
}

for name, model in models.items():
    score = cross_val_score(model, X, Y, cv=5).mean()
    print(f"{name}: Accuracy = {score:.4f}")


Linear SVM: Accuracy = 0.7735
RBF SVM: Accuracy = 0.7709
KNN: Accuracy = 0.7331
Neural Net: Accuracy = 0.7618


Training the Model

In [80]:
classifier = svm.SVC(kernel='linear')

In [81]:
#training the support vector Machine Classifier
classifier.fit(X_train, Y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


Model Evaluation

Accuracy Score

In [82]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [83]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7866449511400652


In [84]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [85]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7727272727272727


Making a Predictive System

In [86]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 0.3429808   1.41167241  0.14964075 -0.09637905  0.82661621 -0.78595734
   0.34768723  1.51108316]]
[1]
The person is diabetic


D:\AI\Practice\AIEnvironments\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\AI\Practice\AIEnvironments\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
